In [3]:
import os
import torch

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm
import timm
from sklearn.model_selection import train_test_split

# Configuration
class Config:
    DATA_PATH = '../input/paddy-disease-classification'  # Kaggle dataset path
    TRAIN_PATH = f'{DATA_PATH}/train_images'
    TEST_PATH = f'{DATA_PATH}/test_images'
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    NUM_CLASSES = 15
    IMAGE_SIZE = 224
    BATCH_SIZE = 32
    EPOCHS = 20
    LEARNING_RATE = 2e-4
    WEIGHT_DECAY = 1e-4

# Dataset class
class PaddyDataset(Dataset):
    def __init__(self, image_paths, labels=None, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.labels is not None:
            return image, self.labels[idx]
        return image

# Model class
class PaddyModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)
        
    def forward(self, x):
        return self.model(x)

# Data transformations
def get_transforms(is_train=True):
    if is_train:
        return transforms.Compose([
            transforms.RandomResizedCrop(Config.IMAGE_SIZE),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize((Config.IMAGE_SIZE, Config.IMAGE_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

# Training function
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for images, labels in tqdm(loader, desc='Training'):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return total_loss/len(loader), 100.*correct/total

# Validation function
@torch.no_grad()
def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    for images, labels in tqdm(loader, desc='Validation'):
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return total_loss/len(loader), 100.*correct/total

def main():
    print(f"Using device: {Config.DEVICE}")
    
    # Prepare data
    all_images = []
    labels = []
    class_map = {}
    
    # Load training data
    for idx, class_name in enumerate(sorted(os.listdir(Config.TRAIN_PATH))):
        class_path = os.path.join(Config.TRAIN_PATH, class_name)
        class_map[class_name] = idx
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            all_images.append(img_path)
            labels.append(idx)
    
    # Split data
    X_train, X_val, y_train, y_val = train_test_split(
        all_images, labels, test_size=0.2, stratify=labels, random_state=42
    )
    
    # Create datasets
    train_dataset = PaddyDataset(X_train, y_train, get_transforms(is_train=True))
    val_dataset = PaddyDataset(X_val, y_val, get_transforms(is_train=False))
    
    # Create dataloaders
    train_loader = DataLoader(
        train_dataset, 
        batch_size=Config.BATCH_SIZE,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=Config.BATCH_SIZE,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )
    
    # Initialize model
    model = PaddyModel(Config.NUM_CLASSES).to(Config.DEVICE)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=Config.LEARNING_RATE,
        weight_decay=Config.WEIGHT_DECAY
    )
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.EPOCHS)
    
    # Training loop
    best_val_acc = 0
    
    for epoch in range(Config.EPOCHS):
        print(f"\nEpoch {epoch+1}/{Config.EPOCHS}")
        
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, Config.DEVICE
        )
        
        val_loss, val_acc = validate(
            model, val_loader, criterion, Config.DEVICE
        )
        
        scheduler.step()
        
        print(f"Train Loss: {train_loss:.4f} Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f} Val Acc: {val_acc:.2f}%")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"Saved best model with validation accuracy: {val_acc:.2f}%")
    
    # Prepare submission
    model.load_state_dict(torch.load('best_model.pth'))
    test_images = sorted([
        os.path.join(Config.TEST_PATH, img_name)
        for img_name in os.listdir(Config.TEST_PATH)
    ])
    
    test_dataset = PaddyDataset(test_images, transform=get_transforms(is_train=False))
    test_loader = DataLoader(
        test_dataset,
        batch_size=Config.BATCH_SIZE,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )
    
    # Make predictions
    model.eval()
    predictions = []
    image_ids = []
    
    with torch.no_grad():
        for images in tqdm(test_loader, desc='Predicting'):
            images = images.to(Config.DEVICE)
            outputs = model(images)
            _, preds = outputs.max(1)
            predictions.extend(preds.cpu().numpy())
    
    # Create submission file
    rev_class_map = {v: k for k, v in class_map.items()}
    submission = pd.DataFrame({
        'image_id': [os.path.basename(img_path) for img_path in test_images],
        'label': [rev_class_map[pred] for pred in predictions]
    })
    
    submission.to_csv('submission.csv', index=False)
    print("Submission file created!")

if __name__ == "__main__":
    main()

Using device: cuda

Epoch 1/20


Validation: 100%|██████████| 66/66 [00:28<00:00,  2.33it/s]


Train Loss: 1.7840 Train Acc: 38.57%
Val Loss: 1.3885 Val Acc: 54.03%
Saved best model with validation accuracy: 54.03%

Epoch 2/20


Validation: 100%|██████████| 66/66 [00:28<00:00,  2.32it/s]


Train Loss: 1.1801 Train Acc: 60.46%
Val Loss: 0.8540 Val Acc: 71.18%
Saved best model with validation accuracy: 71.18%

Epoch 3/20


...

Train Loss: 0.1149 Train Acc: 95.88%
Val Loss: 0.1298 Val Acc: 96.54%
Saved best model with validation accuracy: 96.54%

Epoch 20/20


Validation: 100%|██████████| 66/66 [00:28<00:00,  2.33it/s]
<ipython-input-3-6d4dcce7805b>:199: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.loa

Train Loss: 0.1155 Train Acc: 96.12%
Val Loss: 0.1308 Val Acc: 96.49%


Predicting: 100%|██████████| 109/109 [00:47<00:00,  2.29it/s]

Submission file created!
